# CS 109A/AC 209A/STAT 121A Data Science: Lab 10
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Wednesday, November 9th, 2016 at 11:59pm

In [11]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Predicting Outcome of a Fund-raising Campaign, Revisited

In this lab, we revisit a problem we tackled in Homework 4. 

You are provided a data set containing details of mail sent to 95,412 potential donors for a fund-raising campaign of a not-for-profit organization. This data set also contains the amount donated by each donor. The task is to build a model that can identify which donors to mail in order to expected maximize net contribution, given the cost of mailing a flyer or package is \$7 per donor. 

In Homework 4, we used linear regression to first predict the amount that each individual will donate and then mailed a flyer to only those individuals that were predicted to donate more than \$7.

In this lab, we will cast this problem as a classification problem: we build a model to classify each individual as a mail-worthy donor (will likely donate more than \$7) or a un-mail-worthy donor (will likely donate less than \$7). Again, our goal is to maximize the expected net contribution.

The data is contained in the file `dataset.txt`. Each row contains 376 attributes for a donor, followed by the donation amount.

## Step 1: Clean and explore the data

First let's read and explore the data.

In [13]:
#Load and inspect the data
data = pd.read_csv('datasets/dataset.txt', low_memory=False)  # low memory is set false for better type inference
data.head()

,OSOURCE,TCODE,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,TARGET_D
0,BBK,2,MN,_,0,_,_,_,_,XXXX,...,1,L,3,D,X,X,X,3,A,4
1,SYN,0,TX,_,0,_,_,_,_,XXXX,...,1,L,3,D,X,X,X,14,A,7
2,DRK,0,IA,_,0,_,_,_,_,XXXX,...,1,L,3,D,X,X,X,11,C,5
3,BHG,0,CA,_,0,_,_,_,_,XXXX,...,0,L,2,F,X,X,X,2,A,13
4,L01,1,GA,_,0,_,_,_,_,XXXX,...,1,L,3,E,X,X,X,22,A,10


The entries in the dataframe that read `'_'` are missing values. They should be replaced with NaN before you decide what to do with them.

In [14]:
#Replace missing values with NaN
data = data.replace('_', np.nan)
data.head()

,OSOURCE,TCODE,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,TARGET_D
0,BBK,2,MN,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1,L,3,D,X,X,X,3,A,4
1,SYN,0,TX,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1,L,3,D,X,X,X,14,A,7
2,DRK,0,IA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1,L,3,D,X,X,X,11,C,5
3,BHG,0,CA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,0,L,2,F,X,X,X,2,A,13
4,L01,1,GA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1,L,3,E,X,X,X,22,A,10


It looks like the NaN values appear only in a few columns. For the sake of expediency, we will simply drop the columns with missing values. **You might want to handle the missing data in a more sophisticated way**.

In [15]:
#Remove columns with missing values

complete_cols = [column for column in data.columns if len(data[column][data[column].isnull()]) == 0]
        
data = data[complete_cols]

data.head()

,TCODE,STATE,NOEXCH,MDMAUD,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,TARGET_D
0,2,MN,0,XXXX,10,2,25,40,27,11,...,4.066667,1,L,3,D,X,X,X,3,4
1,0,TX,0,XXXX,0,1,37,58,16,8,...,6.181818,1,L,3,D,X,X,X,14,7
2,0,IA,0,XXXX,5,0,33,24,39,6,...,4.857143,1,L,3,D,X,X,X,11,5
3,0,CA,0,XXXX,0,0,34,20,54,2,...,11.000000,0,L,2,F,X,X,X,2,13
4,1,GA,0,XXXX,10,0,21,53,8,5,...,9.400000,1,L,3,E,X,X,X,22,10


We see that some of the predictors are categorical and some are quantitative. We need to convert the real-valued predictor values into floating points and encode the categorical variables as integers (we will furthre convert the categorical variables into binary variable using one-hot-encoding). To decide which variables are quantitative and which are categorical, you need to interpret the meaning of each predictor and use your common sense.

In [16]:
# Categoricals will be int or str (object), the rest float

# List of columns to be converted to floating point
to_float = ['HIT', 'MALEMILI', 'MALEVET', 'VIETVETS', 'WWIIVETS', 'LOCALGOV', 'STATEGOV', 'FEDGOV', 'NUMPRM12', 
           'CARDPM12', 'CARDPROM', 'NUMPROM', 'NGIFTALL', 'CARDGIFT']

# Converted columns to floating point
for feature_name in to_float:
    data[feature_name] = data[feature_name].astype(float)

# Columns between POP901 to AC2 should all be float
index1 = data.columns.get_loc("POP901")
index2 = data.columns.get_loc("AC2")

for i in range(index1, index2 + 1):
    data.iloc[:, i] = data.iloc[:, i].astype(float)

In [17]:
#Encode categorical variables
def encode_categorical(array):
    if not array.dtype == np.dtype('float64'):
        return preprocessing.LabelEncoder().fit_transform(array) 
    else:
        return array
    
# Categorical columns for use in one-hot encoder
categorical = (data.dtypes.values != np.dtype('float64'))

# Encode all labels
data = data.apply(encode_categorical)

# Get numpy array from data
x = data.values[:, :-1]
y = data.values[:, -1]

# Apply one hot endcoing
encoder = preprocessing.OneHotEncoder(categorical_features=categorical[:-1], sparse=False)  # Last value in mask is y
x = encoder.fit_transform(x)

Let's split our dataset into train and test.

In [18]:
x_train, x_test, y_train_val, y_test_val = train_test_split(x, y, test_size=0.6, random_state=42)

Recall that the response variable, amount donated, is a real-valued variable not a binary variable! We need to convert the amount donated into a binary value: 0 for under \$7 and 1 for over \$7.

(***Experimentation suggested***)

In [2]:
#Cost of mailing each potential donor
cost_per_donor = 7

#Threshold for class 0
threshold = 7

#duplicate y_train_val

#duplicate y_test_val

#turn values greater than threshold to 1

#turn values under or equal to threshold to 0

#Print some useful info for our test, train sets

#print x_train shape

#print x_test shape

#print proportion of classes in train 

#print proportion of classes in test

---

## Step 2: Establish the Baseline Models (for Sanity Check)

What are the baseline models in this case? We can check off three basic models: 

1. a model that labels everything 1
2. a model that labels everything 0
3. a model that randomly guesses a label, 1 or 0

Before implementing anything fancy, let's implement these baseline models and see how they do.

**Note:** Again, think about accuracy in a **meaningful** way. Hint: is overall accuracy a great metric?

(***Experimentation suggested***)

In [156]:
#A model that labels everything 1: call it pos_model

    
#A model that labels everything 0: call it neg_model


#A model that randomly labels things: : call it random_model



In [4]:
#Evaluate pos_model using overall accuracy

#Evaluate pos_model using other meaningful classification metrics


#Evaluate neg_model using overall accuracy

#Evaluate neg_model using other meaningful classification metrics


#Evaluate random_model using overall accuracy

#Evaluate random_model using other meaningful classification metrics



---

## Step 3: Build Fancier Models

Now that we have an idea of how baseline models perform, let's try to improve upon them with fancier classifiers.

We've built you a helper function that will take as input a model and the testing data, returning a `pandas` series with the score of your model on the testing data evaluated under three metrics. 

In [ ]:
#Function for computing the accuracy a given model on the entire test set, the accuracy on class 0 in the test set
#and the accuracy on class 1
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])],
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

Implement all the models you know for classification and evaluate them using the `score` function.

(***Experimentation Suggested***)

In [5]:
#KNN


#Unweighted logistic regression


#Weighted logistic regression


#LDA


#QDA


#Decision Tree


#Random Forest


#SVM

We recommend putting the scores for your models together in a dataframe, whose columns titles are the names of the classifiers.

In [6]:
#Score Dataframe

So which model is better? Which out-performs our baseline models? What does "better" mean anyways? 

To perform meaningful model selection, we have to remember our task! What task were you charged with? Does looking at accuracy rates address how well you've accomplished this task?

---

## Step 4: Meaningful Model Evaluation

To meaningully assess the effecitveness of our models, we have to evaluate them according to the utility function that computes the expected net contribution.

In [161]:
#--------  expected_profit
# A function that computes the expected net contribution generated from a mailing scheme based on
# your classification of potential doners as mail-worthy or not mail-worthy
# Input: 
#      y_true_val (the true amount donated by each individual)
#      y_true (true class labels)
#      y_pred (predicted class labels)
# Returns: 
#      expected_profit (expected net contribution)

def expected_profit(y_true_val, y_true, y_pred):
    
    profit = []
    
    for i in range(5000):
    
        sample = np.random.choice(len(y_true_val), len(y_true_val))

        true_donations = y_true_val[sample]
        true_labels = y_true[sample]
        pred_labels = y_pred[sample]

        pred_donors = pred_labels > 0

        cost = (pred_donors).sum() * 7.

        donations = true_donations[pred_donors].sum()

        profit.append(donations - cost)
        
        expected_profit = np.mean(profit)
    
    return expected_profit

Evaluate your models using the `expected_profit` function.

(***Experimentation Suggested***)

In [ ]:
#KNN


#Unweighted logistic regression


#Weighted logistic regression


#LDA


#QDA


#Decision Tree


#Random Forest


#SVM

Again, we recommend keeping your profits in a dataframe.

In [7]:
#Profits Dataframe

How did each model do - which is the best model? 

Compare the net contribution obtained from each model with the accuracy of each model. How is accuracy related to the net contribution? Is accuracy a good predictor of net contribution (meaning is accuracy a meaningful metric for evaluating our models)?

What can we do to improve the performance of our models? 

**Hint:** Would tuning these models help? Consider, also, all the ensemble methods you've explored in the past two Homework sets: boosting and meta-learning.

---

## Step 5: Tuning to Maximize Net Contribution

Take any model from the above, we can try to improve its performance (in terms of maximizing net contribution) by tuning its parameters to maximize the profit function. Here we demonstrate how to tune a decision tree. You should try the same for the rest of the models.

In [19]:
#Set a range of depths
depths = range(1, 20)
#Create a k-folds validation object
kf = KFold(len(x_train), n_folds=5)
#Record the mean profit for each depth
profits = []

#Iterate over the depths

    #Record the profit for each fold

    #Iterate over the folds
        
        #Split x_train into a train and "test" set

        #Fit a decision tree

        #Use your decision tree to predict a set of labels for "test"
        
        #Compute the expected profits of your decision tree model
    
    
    #Append the mean profit over the k-folds to profit list

#Set best depth of tree to the one corresponding to highest mean profit

#Fit a decision tree with optimal depth

#Use your decision tree to predict a set of labels for x_test

#Compute the expected profits of your decision tree model

Now, consider what you can tune for weighted logistic regression (or SVM or random forest). Certainly, there is the regularization parameter $C$. In the above we just weighted each sample by the reciprocal of the class proportion (for the class that the sample belongs to). But can we weight the samples differently (using the `sample_weight` parameter in the `.fit()` function of the `sklearn` models)? How can you weight the samples to achieve maximum profit?

**Hint:** think about how profit is related to accuracy in this application.

---